<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Build-Back--Propagation-with-numpy" data-toc-modified-id="Build-Back--Propagation-with-numpy-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Build Back  Propagation with numpy</a></span></li><li><span><a href="#데이터-불러오기" data-toc-modified-id="데이터-불러오기-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>데이터 불러오기</a></span></li><li><span><a href="#MLP-construct" data-toc-modified-id="MLP-construct-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>MLP construct</a></span><ul class="toc-item"><li><span><a href="#optimizer" data-toc-modified-id="optimizer-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>optimizer</a></span></li><li><span><a href="#성능-평가를-위한-함수-정의" data-toc-modified-id="성능-평가를-위한-함수-정의-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>성능 평가를 위한 함수 정의</a></span></li><li><span><a href="#함수-실행" data-toc-modified-id="함수-실행-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>함수 실행</a></span></li><li><span><a href="#과제" data-toc-modified-id="과제-3.4"><span class="toc-item-num">3.4&nbsp;&nbsp;</span>과제</a></span></li></ul></li></ul></div>

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms, datasets
from math import exp

## Build Back  Propagation with numpy

* 0과 1의 정답 레이블을 가진 데이터 셋에서 0과 1로 분류를 하고 싶다.
* dataset 은 2차원 벡터로 이루어져 있다.
* 정답 레이블은 각각 0과 1로 되어 있다.
* 레이어를 직접 만들어서 코드 구현을 한다.

In [2]:
n_name = ['activate_function','forward_propagate','transfer','update_weights','backward_propagate_error','transfer_derivative']
# 활성화 함수, 순전파, 비선형 결합, 가중치 업데이트, 오차 역전파 에러 계산, 비선형 함수 미분을 의미한다.

In [4]:
from random import seed
from random import random

def initialize_network(n_inputs, n_hidden, n_outputs):

    network = [] 

    hidden_layer = [{'weights':[random() for i in range(n_inputs)]} for i in range(n_hidden)]
    # hidden layer는 input에 대해 0~1 사이의 무작위 가중치를 가지고, n_hidden만큼 반복한다.
    
    bias = random() 
    for j in range(n_hidden): 
        hidden_layer[j]['weights'].append(bias) 
    print('hidden : ' , hidden_layer) 
    # hidden layer의 각 가중치에 무작위로 생성한 편향을 추가하고, hidden layer를 출력한다.
    
    network.append(hidden_layer) 
    
    output_layer = [{'weights':[random() for i in range(n_hidden)]} for i in range(n_outputs)]  
    # output layer는 hidden에 대해 0~1 사이의 무작위 가중치를 가지고, n_outputs만큼 반복한다.
    
    bias = random() 
    for j in range(n_outputs): 
        output_layer[j]['weights'].append(bias)
    print('output : ', output_layer)
    # output layer의 각 가중치에 대해 무작위로 생성한 편향을 추가하고, output layer를 출력한다.

    network.append(output_layer) 
    
    return network 
 
seed(1) 
network = initialize_network(2, 1, 2) 

hidden :  [{'weights': [0.13436424411240122, 0.8474337369372327, 0.763774618976614]}]
output :  [{'weights': [0.2550690257394217, 0.4494910647887381]}, {'weights': [0.49543508709194095, 0.4494910647887381]}]


In [4]:
def a(weights, inputs): 
    a_1 = weights[-1] 
    for i in range(len(weights)-1): 
        a_1 += weights[i] * inputs[i] 
    return a_1        # 가중치 * 입력값 + 편향을 a_1에 저장

def b(a_1): 
    return 1.0 / (1.0 + exp(-a_1))  #시그모이드 함수


In [5]:
def c(network, row):    # 순전파 함수
    inputs = row 
    for layer in network:      # 네트워크의 각 레이어에 대한 반복
        new_inputs = [] 
        
        for neuron in layer:   # 현재 층의 각 뉴런에 대해 반복
            a_1 = a(neuron['weights'], inputs)    # 뉴런의 가중치 * 입력값
            neuron['output'] = b(a_1)     # 시그모이드 함수를 적용후 뉴런의 output에 저장
            
            new_inputs.append(neuron['output']) #새로운 입력값에 뉴런의 출력값을 저장
        inputs = new_inputs 
 
    return inputs 

In [6]:
network

[[{'weights': [0.13436424411240122, 0.8474337369372327, 0.763774618976614]}],
 [{'weights': [0.2550690257394217, 0.4494910647887381]},
  {'weights': [0.49543508709194095, 0.4494910647887381]}]]

In [7]:
def d(output): 
    return output * (1.0 - output) # 시그모이드 함수의 미분 함수를 적용

def e(network, expected):   # 역전파 함수


    for i in reversed(range(len(network))):  # 역순으로 각 층에 대해서 반복
        layer = network[i] 
        errors = [] 
        if i == len(network) -1 :   # output_layer인 경우 
            for j in range(len(layer)): 
                neuron = layer[j] 
                errors.append(2*(neuron['output'] - expected[j]))  # (뉴런의 출력값 - expected값)*2를 통해 오차 계산


            
        else:       # hidden_layer인 경우
            for j in range(len(layer)): 
                error = 0.0 
                for neuron in network[i + 1]:  # 다음 층의 뉴런의 가중치*델타값을 통해 에러 추가
                    error += (neuron['weights'][j] * neuron['delta']) 
                errors.append(error) 
        for j in range(len(layer)): 
            neuron = layer[j] 
            neuron['delta'] = errors[j] * d(neuron['output']) # 각 층의 뉴런의 델타값을 오차*출력값의 시그모이드 함수의 미분함수를 이용하여 구함

         

def f(network, row, l_rate):
    for i in range(len(network)):
        inputs = row[:-1]   # 입력값을 이전층의 출력값을 사용
        if i != 0:
            inputs = [neuron['output'] for neuron in network[i - 1]]   # 각 뉴런에 대해 반복하여 가중치를 업데이트
        for neuron in network[i]:
            for j in range(len(inputs)):
                neuron['weights'][j] -= l_rate * neuron['delta'] * inputs[j] # 각 뉴런의 가중치를 업데이트
            neuron['weights'][-1] -= l_rate * neuron['delta'] # 각 뉴런의 편향에 대해 업데이트
 

In [8]:
def train_network(network, train, l_rate, n_epoch, n_outputs):
    for epoch in range(n_epoch):   # epoch 횟수 만큼 반복 실행
        sum_error = 0
        for row in train:  
            outputs = c(network, row)  # 순전파 계산하여 출력값 저장
            expected = [0 for i in range(n_outputs)] 
            expected[row[-1]] = 1
            sum_error += sum([(expected[i]-outputs[i])**2 for i in range(len(expected))])
            e(network, expected)
            f(network, row, l_rate)  # 역전파를 통해 네트워크의 가중치 업데이트
            
        print('>epoch=%d, lrate=%.3f, error=%.3f' % (epoch, l_rate, sum_error)) 
### 밑 부분 생략 ###
seed(1)
dataset = [[2.7810836,2.550537003,0],
    [1.465489372,2.362125076,0],
    [3.396561688,4.400293529,0],
    [1.38807019,1.850220317,0],
    [3.06407232,3.005305973,0],
    [7.627531214,2.759262235,1],
    [5.332441248,2.088626775,1],
    [6.922596716,1.77106367,1],
    [8.675418651,-0.242068655,1],
    [7.673756466,3.508563011,1]]
n_inputs = len(dataset[0]) - 1   # 입력 노드의 개수 설정
n_outputs = len(set([row[-1] for row in dataset]))  # 출력 노드의 개수 설정
network = initialize_network(n_inputs, 2, n_outputs)  # 신경망을 초기화
train_network(network, dataset, 0.4, 20, n_outputs)  # 신경망 훈련

hidden :  [{'weights': [0.13436424411240122, 0.8474337369372327, 0.49543508709194095]}, {'weights': [0.763774618976614, 0.2550690257394217, 0.49543508709194095]}]
output :  [{'weights': [0.4494910647887381, 0.651592972722763, 0.02834747652200631]}, {'weights': [0.7887233511355132, 0.0938595867742349, 0.02834747652200631]}]
>epoch=0, lrate=0.400, error=5.680
>epoch=1, lrate=0.400, error=5.395
>epoch=2, lrate=0.400, error=5.494
>epoch=3, lrate=0.400, error=5.541
>epoch=4, lrate=0.400, error=5.556
>epoch=5, lrate=0.400, error=5.558
>epoch=6, lrate=0.400, error=5.557
>epoch=7, lrate=0.400, error=5.554
>epoch=8, lrate=0.400, error=5.551
>epoch=9, lrate=0.400, error=5.548
>epoch=10, lrate=0.400, error=5.543
>epoch=11, lrate=0.400, error=5.538
>epoch=12, lrate=0.400, error=5.532
>epoch=13, lrate=0.400, error=5.525
>epoch=14, lrate=0.400, error=5.515
>epoch=15, lrate=0.400, error=5.501
>epoch=16, lrate=0.400, error=5.481
>epoch=17, lrate=0.400, error=5.449
>epoch=18, lrate=0.400, error=5.395
>

In [9]:
print(n_inputs);network

2


[[{'weights': [0.41439217415583623, -0.07372576040876247, 0.2239514103983687],
   'output': 0.9466777040134029,
   'delta': -0.009301704985056515},
  {'weights': [0.8487307648745676, 0.39942679445456875, 0.5629167140896112],
   'output': 0.9997918685040618,
   'delta': -5.251383770140929e-06}],
 [{'weights': [-0.33852568729014876,
    -0.09209047896314558,
    -0.33797731646752127],
   'output': 0.3647706726777963,
   'delta': 0.1690442349364133},
  {'weights': [0.9314012439824348, 0.1966269128954129, -0.26904650637316496],
   'output': 0.6515912036010892,
   'delta': -0.1581916044700144}]]

## 데이터 불러오기

In [5]:
from torchvision.datasets import MNIST

In [6]:
if torch.cuda.is_available():
  DEVICE = torch.device('mps')
else:
  DEVICE = torch.device('cpu')

In [19]:
BATCH_SIZE = 50
EPOCHS = 50

In [20]:
train_dataset = datasets.MNIST(root = "../data/MNIST", # 데이터 지정 장소
                              train = True,
                              download = True,
                              transform  = transforms.ToTensor() # 데이터의 형태를 텐서로 변환!
                               # 기존 0~255의 Sclar 를 0~1로 정규화
                              )
test_dataset = datasets.MNIST(root = "../data/MNIST",
                              train = False,
                              transform = transforms.ToTensor(),
                              )
train_loader = torch.utils.data.DataLoader(dataset = train_dataset,
                                          batch_size = BATCH_SIZE,
                                          shuffle = True)
test_loader = torch.utils.data.DataLoader(dataset = test_dataset,
                                         batch_size = BATCH_SIZE,
                                         shuffle = False) # 데이터의 순서를 섞기 LABEL 의 순서 정보 무력화

In [21]:
for (X_train, y_train) in train_loader:
    print(X_train.size(), X_train.type())
    print(y_train.size(),y_train.type())
    break
# 여기서 32, 1, 28, 28 의 의미는?

torch.Size([50, 1, 28, 28]) torch.FloatTensor
torch.Size([50]) torch.LongTensor


## MLP construct

$y = xA^T + bias$ <- nn.Linear 함수

In [22]:
class Net(nn.Module):
    def __init__(self):# 인스턴스 생성 시 성질 정의
        super(Net,self).__init__() # Net 내의 메소드 상속받아 이용 가능
        self.fc1 = nn.Linear(28*28,512) # fc1 이라는 객체가 어떤 값인지 반환 28*28의 input data
        self.fc2 = nn.Linear(512, 256) # nn.Linear ->  선형 변환을 시켜주는 함수.
        self.fc3 = nn.Linear(256,10)
    def forward(self,x): # train 안의 forward 를 override  
        x = x.view(-1,28*28) # x Vector 의 size 를 [1,756] 의 열벡터로 변환 # -1은 자동으로 크기를 맞춰준다.
        x = self.fc1(x) # 1X 756 product 756 X 512 -> 1 X 512 에서 가중치 정보를 담는 열벡터 생성 wx + b 의 값
        x = F.sigmoid(x) # sigmoid 함수 통과 (sigma(wx+b))
        x = self.fc2(x) 
        x = F.sigmoid(x)
        x = self.fc3(x)
        x = F.log_softmax(x , dim=1) # dim 1 이라는 것은 각 샘플에 대해서 나열한다는 것. 두번째 차원에 대해서 log_softmax 로 묶음!
        return x

### optimizer

In [23]:
model = Net().to(DEVICE)
optimizer = torch.optim.SGD(model.parameters(), lr = 0.1) #optim 객체 생성
# 경사 하강법 적용을 어떤 식으로 할 것인가??
criterion = nn.CrossEntropyLoss()
print(model)

Net(
  (fc1): Linear(in_features=784, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=256, bias=True)
  (fc3): Linear(in_features=256, out_features=10, bias=True)
)


### 성능 평가를 위한 함수 정의

In [24]:
def train(model, train_loader, optimizer , log_interval):
    model.train() # nn.Module 에서 상속받은 train 이라는 메소드를 사용. 이 train 은 forward 를 진행하게 되는데, 그 forward 과정을 명시적으로 우리가 지정함
    for batch_idx,(image,label) in enumerate(train_loader):
        image = image.to(DEVICE)
        label = label.to(DEVICE)
        optimizer.zero_grad() # 미니배치당 새로운 그레디언트를 부여하기 위해 기울기 초기화 
        output = model(image) # model 에 image 를 넣어 MLP 를 거치게 함
        loss = criterion(output,label) # 우리의 비용함수 정의
        loss.backward()  # Backpropagation 을 통해 계산된 Gradient 값을 할당
        optimizer.step() # 한 스텝씩 움직인다!
        if batch_idx % log_interval == 0: # 나머지가 0 이면 중간 값 출력
            print("Train Epoch: {} [{}/ {}({:.0f}%)]\tTrain Loss:{:6f}".format(Epoch,batch_idx*len(image),
                                                                              len(train_loader.dataset),
                                                                              100.*batch_idx/len(train_loader),
                                                                              loss.item()))

In [25]:
 for batch_idx,(image,label) in enumerate(train_loader):
        print(batch_idx, (image,label))
        break
# 0번째 배치, 32개의 이미지 데이터와 32개의 라벨데이터가 존재
# 32개씩(미니배치 사이즈) 6만개 train을 나누면 나머지가 남는다. 남는 경우? padding 사용해서 처리한다.

0 (tensor([[[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]]],


        [[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]]],


        [[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]]],


        ...,


        [[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ...,

In [26]:
def evaluate(model,test_loader): # 평가를 할때는 그레디언트를 기록할 필요가 없다.
    model.eval() # or model.train(False)
    test_loss = 0
    correct = 0
    with torch.no_grad(): # gradient tracking 작업하는 것을 끄며 메모리 아낌
        for image,label in test_loader:
            image = image.to(DEVICE) # 이미지를 GPU 를 통해 계산하도록 조정
            label = label.to(DEVICE)
            output = model(image) # 모델에서 image 를 판별
            test_loss += criterion(output,label).item()
            prediction = output.max(1, keepdim = True)[1] # 0번째 인덱스는 type 호출이기에 [1] 입력
            correct += prediction.eq(label.view_as(prediction)).sum().item() # dict 형태에서 맞은 값 저장
    test_loss /= len(test_loader.dataset)
    test_accuracy = 100.*correct / len(test_loader.dataset)
    return test_loss,test_accuracy

### 함수 실행

In [27]:
for Epoch in range(1, EPOCHS +1):
    train(model, train_loader, optimizer , log_interval = 200)
    test_loss, test_accuracy = evaluate(model, test_loader)
    print("\n[EPOCH:{}].  \nTest Loss: {:.4f}, \t Test Accuracy: {:.2f}% \n".format(Epoch,test_loss,test_accuracy))
    # batch 32, log_interval = 200 -> 6400개 씩 학습

Train Epoch: 1 [0/ 60000(0%)]	Train Loss:2.338087
Train Epoch: 1 [10000/ 60000(17%)]	Train Loss:2.354712
Train Epoch: 1 [20000/ 60000(33%)]	Train Loss:2.269845
Train Epoch: 1 [30000/ 60000(50%)]	Train Loss:1.928630
Train Epoch: 1 [40000/ 60000(67%)]	Train Loss:1.160651
Train Epoch: 1 [50000/ 60000(83%)]	Train Loss:1.086976

[EPOCH:1].  
Test Loss: 0.0149, 	 Test Accuracy: 77.33% 

Train Epoch: 2 [0/ 60000(0%)]	Train Loss:0.628146
Train Epoch: 2 [10000/ 60000(17%)]	Train Loss:0.934949
Train Epoch: 2 [20000/ 60000(33%)]	Train Loss:0.568266
Train Epoch: 2 [30000/ 60000(50%)]	Train Loss:0.390949
Train Epoch: 2 [40000/ 60000(67%)]	Train Loss:0.741856
Train Epoch: 2 [50000/ 60000(83%)]	Train Loss:0.767928

[EPOCH:2].  
Test Loss: 0.0086, 	 Test Accuracy: 86.36% 

Train Epoch: 3 [0/ 60000(0%)]	Train Loss:0.362522
Train Epoch: 3 [10000/ 60000(17%)]	Train Loss:0.258259
Train Epoch: 3 [20000/ 60000(33%)]	Train Loss:0.438309
Train Epoch: 3 [30000/ 60000(50%)]	Train Loss:0.458645
Train Epoch: 3 [4

Train Epoch: 22 [10000/ 60000(17%)]	Train Loss:0.187383
Train Epoch: 22 [20000/ 60000(33%)]	Train Loss:0.113032
Train Epoch: 22 [30000/ 60000(50%)]	Train Loss:0.129957
Train Epoch: 22 [40000/ 60000(67%)]	Train Loss:0.141096
Train Epoch: 22 [50000/ 60000(83%)]	Train Loss:0.152034

[EPOCH:22].  
Test Loss: 0.0026, 	 Test Accuracy: 96.08% 

Train Epoch: 23 [0/ 60000(0%)]	Train Loss:0.087089
Train Epoch: 23 [10000/ 60000(17%)]	Train Loss:0.100706
Train Epoch: 23 [20000/ 60000(33%)]	Train Loss:0.038550
Train Epoch: 23 [30000/ 60000(50%)]	Train Loss:0.083123
Train Epoch: 23 [40000/ 60000(67%)]	Train Loss:0.200816
Train Epoch: 23 [50000/ 60000(83%)]	Train Loss:0.355398

[EPOCH:23].  
Test Loss: 0.0025, 	 Test Accuracy: 96.18% 

Train Epoch: 24 [0/ 60000(0%)]	Train Loss:0.062517
Train Epoch: 24 [10000/ 60000(17%)]	Train Loss:0.063239
Train Epoch: 24 [20000/ 60000(33%)]	Train Loss:0.023206
Train Epoch: 24 [30000/ 60000(50%)]	Train Loss:0.060964
Train Epoch: 24 [40000/ 60000(67%)]	Train Loss:0.0

Train Epoch: 43 [10000/ 60000(17%)]	Train Loss:0.083503
Train Epoch: 43 [20000/ 60000(33%)]	Train Loss:0.087743
Train Epoch: 43 [30000/ 60000(50%)]	Train Loss:0.028696
Train Epoch: 43 [40000/ 60000(67%)]	Train Loss:0.105104
Train Epoch: 43 [50000/ 60000(83%)]	Train Loss:0.038536

[EPOCH:43].  
Test Loss: 0.0017, 	 Test Accuracy: 97.21% 

Train Epoch: 44 [0/ 60000(0%)]	Train Loss:0.021897
Train Epoch: 44 [10000/ 60000(17%)]	Train Loss:0.042812
Train Epoch: 44 [20000/ 60000(33%)]	Train Loss:0.037237
Train Epoch: 44 [30000/ 60000(50%)]	Train Loss:0.157349
Train Epoch: 44 [40000/ 60000(67%)]	Train Loss:0.029254
Train Epoch: 44 [50000/ 60000(83%)]	Train Loss:0.014344

[EPOCH:44].  
Test Loss: 0.0017, 	 Test Accuracy: 97.36% 

Train Epoch: 45 [0/ 60000(0%)]	Train Loss:0.023605
Train Epoch: 45 [10000/ 60000(17%)]	Train Loss:0.009687
Train Epoch: 45 [20000/ 60000(33%)]	Train Loss:0.030422
Train Epoch: 45 [30000/ 60000(50%)]	Train Loss:0.031903
Train Epoch: 45 [40000/ 60000(67%)]	Train Loss:0.0

In [ ]:
# batch-size: 32, epochs: 50, lr: 0.01로 두었을때는 기존의 Test Accuracy보다 살짝 낮은 95%정도가 나타났었는데
# batch-size:50, epchs: 50, ,lr:0.1로 두었을때는 기존의 Test Accuracy와 동일한 97.59%가 나옴

### 과제
* numpy로 구현한 Backpropagation 코드 각 함수 별 어떤 의미가 있는지 주석 달아보기
* MLP 모델에서 SGD optim 의 paramter, Bath-size, epoch 조정 해서 성능 올려보기